In [ ]:
%run ../notebook_preamble.ipy

In [ ]:
import pyproj
import geopandas as gpd
import shapely
import os

shapely.speedups.enable()

## DEFRA Data Collection

Modelled data for air pollution across the UK is compiled by DEFRA. The values are obtained by using the data from monitoring stations and using atmospheric modelling to interpolate the data to a 1km by 1km grid across the whole country.

## PM10

In [ ]:
years = range(2007, 2019)
base_url = 'https://uk-air.defra.gov.uk/datastore/pcm/mappm10{}g.csv'

In [ ]:
df_year = pd.read_csv(base_url.format(year), header=5, na_values='MISSING')

In [ ]:
for i, year in enumerate(years):
    df_year = pd.read_csv(base_url.format(year), header=5, na_values='MISSING')
    df_year.rename(columns={f'pm10{year}g': year}, inplace=True)
    if i == 0:
        df = df_year
    else:
        df[year] = df_year[year]
    
df = df.dropna()

In [ ]:
df.head()

To match up with shapefiles from Eurostat, we need to convert the UK grid Coordinates (BNG) to decimal lat long coordinates.

In [ ]:
bng = pyproj.Proj('epsg:27700')
wgs84 = pyproj.Proj('epsg:4326')

df['lat'], df['lon'] = pyproj.transform(bng, wgs84, df['x'].values, df['y'].values)

In [ ]:
gdf = gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df['lon'], df['lat']))

In [ ]:
gdf.head()

We are going to assign each point in the grid to a NUTS 2 2016 region.

In [ ]:
year = 2016
f'{data_path}/raw/gis/eurostat/NUTS_RG_01M_{year}_4326_LEVL_2.shp/NUTS_RG_01M_{year}_4326_LEVL_2.shp'
eu_regions = region = gpd.read_file(file)
uk_regions = eu_regions[eu_regions['CNTR_CODE'] == 'UK']

In [ ]:
points_in_poly = gpd.sjoin(gdf, uk_regions, op='within')

And finally we aggregate by NUTS region.

In [ ]:
mean_pm10 = points_in_poly.groupby('NUTS_ID')[list(years)].max().reset_index()
max_pm10 = points_in_poly.groupby('NUTS_ID')[list(years)].max().reset_index()
min_pm10 = points_in_poly.groupby('NUTS_ID')[list(years)].min().reset_index()

In [ ]:
os.mkdir(f'{data_path}/processed/defra')

In [ ]:
mean_pm10.to_csv(f'{data_path}/processed/defra/mean_pm10.csv', index=False)
max_pm10.to_csv(f'{data_path}/processed/defra/max_pm10.csv', index=False)
min_pm10.to_csv(f'{data_path}/processed/defra/min_pm10.csv', index=False)